# **Data Collection**

In [ ]:
#importing needed libraries
import numpy as np
import pandas as pd
import requests
import io
!pip install selenium
from selenium import webdriver
import datetime
from bs4 import BeautifulSoup
import time
import regex as re
from collections import defaultdict
from tqdm import tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 69.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.3/399.3 kB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.1 MB/s eta 0:00:00


## Races DataFrame

In [ ]:
races = {'season': [],
        'round': [],
        'circuit_id': [],
        'lat': [],
        'long': [],
        'country': [],
        'date': [],
        'url': []}

for year in list(range(2018,2023)):

    url = 'https://ergast.com/api/f1/{}.json'
    r = requests.get(url.format(year))
    json = r.json()

    for item in json['MRData']['RaceTable']['Races']:
        try:
            races['season'].append(int(item['season']))
        except:
            races['season'].append(None)

        try:
            races['round'].append(int(item['round']))
        except:
            races['round'].append(None)

        try:
            races['circuit_id'].append(item['Circuit']['circuitId'])
        except:
            races['circuit_id'].append(None)

        try:
            races['lat'].append(float(item['Circuit']['Location']['lat']))
        except:
            races['lat'].append(None)

        try:
            races['long'].append(float(item['Circuit']['Location']['long']))
        except:
            races['long'].append(None)

        try:
            races['country'].append(item['Circuit']['Location']['country'])
        except:
            races['country'].append(None)

        try:
            races['date'].append(item['date'])
        except:
            races['date'].append(None)

        try:
            races['url'].append(item['url'])
        except:
            races['url'].append(None)

races = pd.DataFrame(races)
races.head()

,season,round,circuit_id,lat,long,country,date,url
0,2018,1,albert_park,-37.8497,144.96800,Australia,2018-03-25,http://en.wikipedia.org/wiki/2018_Australian_G...
1,2018,2,bahrain,26.0325,50.51060,Bahrain,2018-04-08,http://en.wikipedia.org/wiki/2018_Bahrain_Gran...
2,2018,3,shanghai,31.3389,121.22000,China,2018-04-15,http://en.wikipedia.org/wiki/2018_Chinese_Gran...
3,2018,4,baku,40.3725,49.85330,Azerbaijan,2018-04-29,http://en.wikipedia.org/wiki/2018_Azerbaijan_G...
4,2018,5,catalunya,41.5700,2.26111,Spain,2018-05-13,http://en.wikipedia.org/wiki/2018_Spanish_Gran...


## Results DataFrame

In [ ]:
# append the number of rounds to each season from the races_df

rounds = []
for year in np.array(races.season.unique()):
    rounds.append([year, list(races[races.season == year]['round'])])

# query API

results = {'season': [],
          'round':[],
           'circuit_id':[],
          'driver': [],
           'date_of_birth': [],
           'nationality': [],
          'constructor': [],
          'grid': [],
          'time': [],
          'status': [],
          'points': [],
          'podium': []}

for n in list(range(len(rounds))):
    for i in rounds[n][1]:

        url = 'http://ergast.com/api/f1/{}/{}/results.json'
        r = requests.get(url.format(rounds[n][0], i))
        json = r.json()

        for item in json['MRData']['RaceTable']['Races'][0]['Results']:
            try:
                results['season'].append(int(json['MRData']['RaceTable']['Races'][0]['season']))
            except:
                results['season'].append(None)

            try:
                results['round'].append(int(json['MRData']['RaceTable']['Races'][0]['round']))
            except:
                results['round'].append(None)

            try:
                results['circuit_id'].append(json['MRData']['RaceTable']['Races'][0]['Circuit']['circuitId'])
            except:
                results['circuit_id'].append(None)

            try:
                results['driver'].append(item['Driver']['driverId'])
            except:
                results['driver'].append(None)

            try:
                results['date_of_birth'].append(item['Driver']['dateOfBirth'])
            except:
                results['date_of_birth'].append(None)

            try:
                results['nationality'].append(item['Driver']['nationality'])
            except:
                results['nationality'].append(None)

            try:
                results['constructor'].append(item['Constructor']['constructorId'])
            except:
                results['constructor'].append(None)

            try:
                results['grid'].append(int(item['grid']))
            except:
                results['grid'].append(None)

            try:
                results['time'].append(int(item['Time']['millis']))
            except:
                results['time'].append(None)

            try:
                results['status'].append(item['status'])
            except:
                results['status'].append(None)

            try:
                results['points'].append(int(item['points']))
            except:
                results['points'].append(None)

            try:
                results['podium'].append(int(item['position']))
            except:
                results['podium'].append(None)


results = pd.DataFrame(results)
results.head()

,season,round,circuit_id,driver,date_of_birth,nationality,constructor,grid,time,status,points,podium
0,2018,1,albert_park,vettel,1987-07-03,German,ferrari,3,5373283.0,Finished,25.0,1
1,2018,1,albert_park,hamilton,1985-01-07,British,mercedes,1,5378319.0,Finished,18.0,2
2,2018,1,albert_park,raikkonen,1979-10-17,Finnish,ferrari,2,5379592.0,Finished,15.0,3
3,2018,1,albert_park,ricciardo,1989-07-01,Australian,red_bull,8,5380352.0,Finished,12.0,4
4,2018,1,albert_park,alonso,1981-07-29,Spanish,mclaren,10,5401169.0,Finished,10.0,5


## Driver Standing DataFrame

In [ ]:

driver_standings = {'season': [],
                    'round':[],
                    'driver': [],
                    'driver_points': [],
                    'driver_wins': [],
                   'driver_standings_pos': []}

# query API

for n in list(range(len(rounds))):
    for i in rounds[n][1]:    # iterate through rounds of each year

        url = 'https://ergast.com/api/f1/{}/{}/driverStandings.json'
        r = requests.get(url.format(rounds[n][0], i))
        json = r.json()

        for item in json['MRData']['StandingsTable']['StandingsLists'][0]['DriverStandings']:
            try:
                driver_standings['season'].append(int(json['MRData']['StandingsTable']['StandingsLists'][0]['season']))
            except:
                driver_standings['season'].append(None)

            try:
                driver_standings['round'].append(int(json['MRData']['StandingsTable']['StandingsLists'][0]['round']))
            except:
                driver_standings['round'].append(None)

            try:
                driver_standings['driver'].append(item['Driver']['driverId'])
            except:
                driver_standings['driver'].append(None)

            try:
                driver_standings['driver_points'].append(int(item['points']))
            except:
                driver_standings['driver_points'].append(None)

            try:
                driver_standings['driver_wins'].append(int(item['wins']))
            except:
                driver_standings['driver_wins'].append(None)

            try:
                driver_standings['driver_standings_pos'].append(int(item['position']))
            except:
                driver_standings['driver_standings_pos'].append(None)

driver_standings = pd.DataFrame(driver_standings)

# define lookup function to shift points and number of wins from previous rounds

def lookup (df, team, points):
    df['lookup1'] = df.season.astype(str) + df[team] + df['round'].astype(str)
    df['lookup2'] = df.season.astype(str) + df[team] + (df['round']-1).astype(str)
    new_df = df.merge(df[['lookup1', points]], how = 'left', left_on='lookup2',right_on='lookup1')
    new_df.drop(['lookup1_x', 'lookup2', 'lookup1_y'], axis = 1, inplace = True)
    new_df.rename(columns = {points+'_x': points+'_after_race', points+'_y': points}, inplace = True)
    new_df[points].fillna(0, inplace = True)
    return new_df

driver_standings = lookup(driver_standings, 'driver', 'driver_points')
driver_standings = lookup(driver_standings, 'driver', 'driver_wins')
driver_standings = lookup(driver_standings, 'driver', 'driver_standings_pos')

driver_standings.drop(['driver_points_after_race', 'driver_wins_after_race', 'driver_standings_pos_after_race'],
                      axis = 1, inplace = True)

In [ ]:
driver_standings.tail()

,season,round,driver,driver_points,driver_wins,driver_standings_pos
2110,2022,22,zhou,6.0,0.0,18.0
2111,2022,22,albon,4.0,0.0,19.0
2112,2022,22,latifi,2.0,0.0,20.0
2113,2022,22,de_vries,2.0,0.0,21.0
2114,2022,22,hulkenberg,0.0,0.0,22.0


## Constructor Standings DataFrame

In [ ]:
#constructor_rounds = rounds[8:]

constructor_standings = {'season': [],
                    'round':[],
                    'constructor': [],
                    'constructor_points': [],
                    'constructor_wins': [],
                   'constructor_standings_pos': []}
# query API

for n in list(range(len(rounds))):
    for i in rounds[n][1]:

        url = 'https://ergast.com/api/f1/{}/{}/constructorStandings.json'
        r = requests.get(url.format(rounds[n][0], i))
        json = r.json()

        for item in json['MRData']['StandingsTable']['StandingsLists'][0]['ConstructorStandings']:
            try:
                constructor_standings['season'].append(int(json['MRData']['StandingsTable']['StandingsLists'][0]['season']))
            except:
                constructor_standings['season'].append(None)

            try:
                constructor_standings['round'].append(int(json['MRData']['StandingsTable']['StandingsLists'][0]['round']))
            except:
                constructor_standings['round'].append(None)

            try:
                constructor_standings['constructor'].append(item['Constructor']['constructorId'])
            except:
                constructor_standings['constructor'].append(None)

            try:
                constructor_standings['constructor_points'].append(int(item['points']))
            except:
                constructor_standings['constructor_points'].append(None)

            try:
                constructor_standings['constructor_wins'].append(int(item['wins']))
            except:
                constructor_standings['constructor_wins'].append(None)

            try:
                constructor_standings['constructor_standings_pos'].append(int(item['position']))
            except:
                constructor_standings['constructor_standings_pos'].append(None)

constructor_standings = pd.DataFrame(constructor_standings)

constructor_standings = lookup(constructor_standings, 'constructor', 'constructor_points')
constructor_standings = lookup(constructor_standings, 'constructor', 'constructor_wins')
constructor_standings = lookup(constructor_standings, 'constructor', 'constructor_standings_pos')

constructor_standings.drop(['constructor_points_after_race', 'constructor_wins_after_race','constructor_standings_pos_after_race' ],
                           axis = 1, inplace = True)

In [ ]:
constructor_standings.tail()

,season,round,constructor,constructor_points,constructor_wins,constructor_standings_pos
1025,2022,22,alfa,55.0,0.0,6.0
1026,2022,22,aston_martin,50.0,0.0,7.0
1027,2022,22,haas,37.0,0.0,8.0
1028,2022,22,alphatauri,35.0,0.0,9.0
1029,2022,22,williams,8.0,0.0,10.0


## Qualifying DataFrame

In [ ]:
qualifying_results = pd.DataFrame()

# Qualifying times are only available from 1983

for year in list(range(2018,2023)):
    url = 'https://www.formula1.com/en/results.html/{}/races.html'
    r = requests.get(url.format(year))
    soup = BeautifulSoup(r.text, 'html.parser')

    # find links to all circuits for a certain year

    year_links = []
    for page in soup.find_all('a', attrs = {'class':"resultsarchive-filter-item-link FilterTrigger"}):
        link = page.get('href')
        if f'/en/results.html/{year}/races/' in link:
            year_links.append(link)

    # for each circuit, switch to the starting grid page and read table

    year_df = pd.DataFrame()
    new_url = 'https://www.formula1.com{}'
    for n, link in list(enumerate(year_links)):
        link = link.replace('race-result.html', 'starting-grid.html')
        df = pd.read_html(new_url.format(link))
        df = df[0]
        df['season'] = year
        df['round'] = n+1
        for col in df:
            if 'Unnamed' in col:
                df.drop(col, axis = 1, inplace = True)

        year_df = pd.concat([year_df, df])

    # concatenate all tables from all years

    qualifying_results = pd.concat([qualifying_results, year_df])

# rename columns

qualifying_results.rename(columns = {'Pos': 'grid', 'Driver': 'driver_name', 'Car': 'constructor',
                                     'Time': 'qualifying_time'}, inplace = True)
# drop driver number column

qualifying_results.drop('No', axis = 1, inplace = True)

In [ ]:
qualifying_results.tail()

,grid,driver_name,constructor,qualifying_time,season,round
15,16,Kevin Magnussen MAG,Haas Ferrari,1:25.834,2022,22
16,17,Pierre Gasly GAS,AlphaTauri RBPT,1:25.859,2022,22
17,18,Valtteri Bottas BOT,Alfa Romeo Ferrari,1:25.892,2022,22
18,19,Alexander Albon ALB,Williams Mercedes,1:26.028,2022,22
19,20,Nicholas Latifi LAT,Williams Mercedes,1:26.054,2022,22


## Weather DataFrame

In [ ]:
weather = races.iloc[:,[0,1,2]] #data frame 2- weather

info = []

# read wikipedia tables

for link in races.url : #and f1_fan_urls :
    try:
        df = pd.read_html(link)[0]
        if 'Weather' in list(df.iloc[:,0]):
            n = list(df.iloc[:,0]).index('Weather')
            info.append(df.iloc[n,1])
        else:
            df = pd.read_html(link)[1]
            if 'Weather' in list(df.iloc[:,0]):
                n = list(df.iloc[:,0]).index('Weather')
                info.append(df.iloc[n,1])
            else:
                df = pd.read_html(link)[2]
                if 'Weather' in list(df.iloc[:,0]):
                    n = list(df.iloc[:,0]).index('Weather')
                    info.append(df.iloc[n,1])
                else:
                    df = pd.read_html(link)[3]
                    if 'Weather' in list(df.iloc[:,0]):
                        n = list(df.iloc[:,0]).index('Weather')
                        info.append(df.iloc[n,1])
                    else:
                        driver = webdriver.Chrome()
                        driver.get(link)

                        # click language button
                        button = driver.find_element_by_link_text('Italiano')
                        button.click()

                        # find weather in italian with selenium

                        clima = driver.find_element_by_xpath('//*[@id="mw-content-text"]/div/table[1]/tbody/tr[9]/td').text
                        info.append(clima)

    except:
        info.append('not found')

# append column with weather information to dataframe

weather['weather'] = info

# set up a dictionary to convert weather information into keywords

weather_dict = {'weather_warm': ['soleggiato', 'clear', 'warm', 'hot', 'sunny', 'fine', 'mild', 'sereno'],
               'weather_cold': ['cold', 'fresh', 'chilly', 'cool'],
               'weather_dry': ['dry', 'asciutto'],
               'weather_wet': ['showers', 'wet', 'rain', 'pioggia', 'damp', 'thunderstorms', 'rainy'],
               'weather_cloudy': ['overcast', 'nuvoloso', 'clouds', 'cloudy', 'grey', 'coperto']}

# map new df according to weather dictionary

weather_df = pd.DataFrame(columns = weather_dict.keys())
for col in weather_df:
    weather_df[col] = weather['weather'].map(lambda x: 1 if any(i in weather_dict[col] for i in x.lower().split()) else 0)

weather_info = pd.concat([weather, weather_df], axis = 1)

<ipython-input-10-ac955db65987>:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather['weather'] = info


In [ ]:
weather_info.tail()

,season,round,circuit_id,weather,weather_warm,weather_cold,weather_dry,weather_wet,weather_cloudy
98,2022,18,suzuka,Rain,0,0,0,1,0
99,2022,19,americas,Partly cloudy,0,0,0,0,1
100,2022,20,rodriguez,Partly cloudy,0,0,0,0,1
101,2022,21,interlagos,Cloudy,0,0,0,0,1
102,2022,22,yas_marina,Clear,1,0,0,0,0


## Circuits DataFrame

In [ ]:
# query API

circuits = defaultdict(list)

for year in results.season.unique():
    url = f'http://ergast.com/api/f1/{year}/circuits.json'
    r = requests.get(url)
    json = r.json()

    try:
        items = json['MRData']['CircuitTable']['Circuits']
        for j in range(len(items)):

            try:
                circuits['circuitId'].append(items[j]['circuitId'])
            except:
                circuits['circuitId'].append(np.nan)

            try:
                circuits['circuitName'].append(items[j]['circuitName'])
            except:
                circuits['circuitName'].append(np.nan)

            try:
                circuits['lat'].append(items[j]['Location']['lat'])
            except:
                circuits['lat'].append(np.nan)

            try:
                circuits['long'].append(items[j]['Location']['long'])
            except:
                circuits['long'].append(np.nan)

            try:
                circuits['locality'].append(items[j]['Location']['locality'])
            except:
                circuits['locality'].append(np.nan)

            try:
                circuits['country'].append(items[j]['Location']['country'])
            except:
                circuits['country'].append(np.nan)

            try:
                circuits['url'].append(items[j]['url'])
            except:
                circuits['url'].append(np.nan)

    except:
        pass

circuits = pd.DataFrame(circuits)
circuits = circuits.drop_duplicates().reset_index(drop=True)
circuits

,circuitId,circuitName,lat,long,locality,country,url
0,albert_park,Albert Park Grand Prix Circuit,-37.8497,144.968,Melbourne,Australia,http://en.wikipedia.org/wiki/Melbourne_Grand_P...
1,americas,Circuit of the Americas,30.1328,-97.6411,Austin,USA,http://en.wikipedia.org/wiki/Circuit_of_the_Am...
2,bahrain,Bahrain International Circuit,26.0325,50.5106,Sakhir,Bahrain,http://en.wikipedia.org/wiki/Bahrain_Internati...
3,baku,Baku City Circuit,40.3725,49.8533,Baku,Azerbaijan,http://en.wikipedia.org/wiki/Baku_City_Circuit
4,catalunya,Circuit de Barcelona-Catalunya,41.57,2.26111,Montmeló,Spain,http://en.wikipedia.org/wiki/Circuit_de_Barcel...
5,hockenheimring,Hockenheimring,49.3278,8.56583,Hockenheim,Germany,http://en.wikipedia.org/wiki/Hockenheimring
6,hungaroring,Hungaroring,47.5789,19.2486,Budapest,Hungary,http://en.wikipedia.org/wiki/Hungaroring
7,interlagos,Autódromo José Carlos Pace,-23.7036,-46.6997,São Paulo,Brazil,http://en.wikipedia.org/wiki/Aut%C3%B3dromo_Jo...
8,marina_bay,Marina Bay Street Circuit,1.2914,103.864,Marina Bay,Singapore,http://en.wikipedia.org/wiki/Marina_Bay_Street...
9,monaco,Circuit de Monaco,43.7347,7.42056,Monte-Carlo,Monaco,http://en.wikipedia.org/wiki/Circuit_de_Monaco


In [ ]:
results.columns

Index(['season', 'round', 'circuit_id', 'driver', 'date_of_birth',
       'nationality', 'constructor', 'grid', 'time', 'status', 'points',
       'podium'],
      dtype='object')

In [ ]:
circuits.circuitId

0        albert_park
1           americas
2            bahrain
3               baku
4          catalunya
5     hockenheimring
6        hungaroring
7         interlagos
8         marina_bay
9             monaco
10             monza
11     red_bull_ring
12            ricard
13         rodriguez
14          shanghai
15       silverstone
16             sochi
17               spa
18            suzuka
19        villeneuve
20        yas_marina
21             imola
22          istanbul
23           mugello
24       nurburgring
25          portimao
26            jeddah
27            losail
28         zandvoort
29             miami
Name: circuitId, dtype: object

In [ ]:
def extractor(line, i):
    try:
        return line[i]
    except:
        return np.nan

In [ ]:
scraped = defaultdict(list)

url = 'https://en.wikipedia.org/wiki/List_of_Formula_One_circuits'
result = requests.get(url)
soup = BeautifulSoup(result.text, 'html.parser')
tables = soup.find_all('table', attrs={'class':'wikitable'})
table = tables[1]
body = table.find('tbody')
for row in body.find_all('tr'):
    row_info = []
    line = [r.text.strip('\n').strip('✔') for r in row.find_all('td') if r.text.strip('\n')!='']

    scraped['name'].append(extractor(line, 0))
    track_type = extractor(line, 1)
    try:
        track_type.strip('circuit').strip(' ')
    except:
        pass
    scraped['type'].append(track_type)
    scraped['direction'].append(extractor(line, 2))
    scraped['location'].append(extractor(line, 3))
    scraped['country'].append(extractor(line, 4))
    scraped['length'].append(extractor(line, 5))

In [ ]:
circuits_info = pd.DataFrame(scraped)
circuits_info.drop([0], inplace=True)
circuits_info.reset_index(drop=True, inplace=True)
circuits_info.head()

,name,type,direction,location,country,length
0,Adelaide Street Circuit,Street circuit,Clockwise,Adelaide,Australia,3.780 km (2.349 mi)
1,Ain-Diab Circuit,Road circuit,Clockwise,Casablanca,Morocco,7.618 km (4.734 mi)
2,Aintree Motor Racing Circuit,Road circuit,Clockwise,Aintree,United Kingdom,4.828 km (3.000 mi)
3,Albert Park Circuit,Street circuit,Clockwise,Melbourne,Australia,5.278 km (3.280 mi)
4,Algarve International Circuit,Race circuit,Clockwise,Portimão,Portugal,4.653 km (2.891 mi)


In [ ]:
def length_formatter(length):
    return float(length.split('km')[0].strip(' ').strip('\xa0'))

circuits_info.length = circuits_info.length.apply(length_formatter)
circuits_info.length.unique()

array([ 3.78 ,  7.618,  4.828,  5.278,  4.653,  4.36 ,  4.909,  4.304,
        5.031,  5.245,  4.309,  4.259,  5.793,  8.3  ,  5.412,  6.003,
        3.703,  5.141,  4.43 ,  3.65 ,  8.055,  7.208,  4.657,  3.337,
        4.411,  6.316,  8.302,  7.004,  3.886,  4.361,  4.265,  5.513,
        5.842,  4.262,  7.775,  5.44 ,  4.428,  3.404,  3.724,  3.901,
        4.168,  4.02 ,  4.563,  4.574,  4.381,  4.192,  5.338,  6.174,
        5.615,  4.261,  6.12 ,  3.275,  5.418,  5.063,  3.791,  3.957,
        5.148, 25.8  ,  3.72 ,  3.92 ,  4.318,  5.271,  6.542,  4.031,
        8.356,  5.543,  5.451,  5.891,  5.848,  5.807,  5.419,  5.43 ,
        5.281,  3.186])

In [ ]:
def referencer(name):
    name = name.lower()

    if 'adelaide' in name:
        return 'adelaide'
    elif 'diab' in name:
        return 'ain-diab'
    elif 'aintree' in name:
        return 'aintree'
    elif 'albert' in name:
        return 'albert_park'
    elif 'americas' in name:
        return 'americas'
    elif 'scandinavian' in name:
        return 'anderstorp'
    elif 'avus' in name:
        return 'avus'
    elif 'baku' in name:
        return 'BAK'
    elif 'bahrain' in name:
        return 'bahrain'
    elif 'boavista' in name:
        return 'boavista'
    elif 'brands hatch' in name:
        return 'brands_hatch'
    elif 'bremgarten' in name:
        return 'bremgarten'
    elif 'buddh' in name:
        return 'buddh'
    elif 'catalunya' in name:
        return 'catalunya'
    elif 'charade' in name:
        return 'charade'
    elif ('fair park' in name) or ('dallas' in name):
        return 'dallas'
    elif 'detroit' in name:
        return 'detroit'
    elif 'dijon' in name:
        return 'dijon'
    elif 'donington' in name:
        return 'donington'
    elif 'essarts' in name:
        return 'essarts'
    elif 'estoril' in name:
        return 'estoril'
    elif 'fuji' in name:
        return 'fuji'
    elif 'juan g' in name:
        return 'galvez'
    elif 'prince george' in name:
        return 'george'
    elif 'hanoi' in name:
        return 'hanoi'
    elif 'hockenheimring' in name:
        return 'hockenheimring'
    elif 'hungaroring' in name:
        return 'hungaroring'
    elif 'imperial' in name:
        return 'port_imperial'
    elif 'dino ferrari' in name:
        return 'imola'
    elif 'indianapolis' in name:
        return 'indianapolis'
    elif 'carlos pace' in name:
        return 'interlagos'
    elif 'istanbul' in name:
        return 'istanbul'
    elif 'jarama' in name:
        return 'jarama'
    elif 'piquet' in name:
        return 'jacarepagua'
    elif 'jeddah' in name:
        return 'jeddah'
    elif 'jerez' in name:
        return 'jerez'
    elif 'kyalami' in name:
        return 'kyalami'
    elif ('las vegas' in name) or ('caesars' in name):
        return 'las_vegas'
    elif ('le mans' in name) or ('bugatti' in name):
        return 'lemans'
    elif 'long beach' in name:
        return 'long_beach'
    elif 'magny' in name:
        return 'magny_cours'
    elif 'marina bay' in name:
        return 'marina_bay'
    elif 'monaco' in name:
        return 'monaco'
    elif 'monsanto' in name:
        return 'monsanto'
    elif 'montju' in name:
        return 'montjuic'
    elif 'monza' in name:
        return 'monza'
    elif 'mosport' in name:
        return 'mosport'
    elif 'mugello' in name:
        return 'mugello'
    elif 'nivelles' in name:
        return 'nivelles'
    elif 'nürburgring' in name:
        return 'nurburgring'
    elif 'okayama' in name:
        return 'okayama'
    elif 'a1' in name:
        return 'osterreichring'
    elif 'pedralbes' in name:
        return 'pedralbes'
    elif 'pescara' in name:
        return 'pescara'
    elif 'phoenix' in name:
        return 'phoenix'
    elif 'algarve' in name:
        return 'portimao'
    elif 'red bull' in name:
        return 'red_bull_ring'
    elif 'reims' in name:
        return 'reims'
    elif 'ricard' in name:
        return 'ricard'
    elif 'riverside' in name:
        return 'riverside'
    elif 'hermanos' in name:
        return 'rodriguez'
    elif 'sebring' in name:
        return 'sebring'
    elif 'sepang' in name:
        return 'sepang'
    elif 'shanghai' in name:
        return 'shanghai'
    elif 'silverstone' in name:
        return 'silverstone'
    elif 'sochi' in name:
        return 'sochi'
    elif 'francorchamps' in name:
        return 'spa'
    elif 'suzuka' in name:
        return 'suzuka'
    elif 'tremblant' in name:
        return 'tremblant'
    elif 'valencia' in name:
        return 'valencia'
    elif 'villeneuve' in name:
        return 'villeneuve'
    elif 'watkins' in name:
        return 'watkins_glen'
    elif 'yas marina' in name:
        return 'yas_marina'
    elif 'korea' in name:
        return 'yeongam'
    elif 'zandvoort' in name:
        return 'zandvoort'
    elif 'zeltweg' in name:
        return 'zeltweg'
    elif 'zolder' in name:
        return 'zolder'
    elif 'miami' in name:
        return 'miami'
    elif 'losail' in name:
        return 'losail'
    else:
        return np.nan


In [ ]:
circuits_info['circuitId'] = circuits_info.name.apply(referencer)
circuits_info.circuitId.unique()
circuits_info.shape

(77, 7)

In [ ]:
circuits_info[circuits_info.circuitId.isnull()]

,name,type,direction,location,country,length,circuitId
8,Autódromo Internacional do Rio de Janeiro,Race circuit,Anti-clockwise,Rio de Janeiro,Brazil,5.031,NaN
11,Autódromo Juan y Oscar Gálvez,Race circuit,Clockwise,Buenos Aires,Argentina,4.259,NaN
72,TI Circuit Aida,Race circuit,Clockwise,Mimasaka,Japan,3.703,NaN


These circuiuts are no longer in use or design since changed and not used as race tracks for the grand prix anymore

In [ ]:
circuits_info.dropna(inplace=True)
circuits_info

,name,type,direction,location,country,length,circuitId
0,Adelaide Street Circuit,Street circuit,Clockwise,Adelaide,Australia,3.780,adelaide
1,Ain-Diab Circuit,Road circuit,Clockwise,Casablanca,Morocco,7.618,ain-diab
2,Aintree Motor Racing Circuit,Road circuit,Clockwise,Aintree,United Kingdom,4.828,aintree
3,Albert Park Circuit,Street circuit,Clockwise,Melbourne,Australia,5.278,albert_park
4,Algarve International Circuit,Race circuit,Clockwise,Portimão,Portugal,4.653,portimao
...,...,...,...,...,...,...,...
71,Suzuka International Racing Course,Race circuit,Part clockwise and part anti-clockwise (figure...,Suzuka,Japan,5.807,suzuka
73,Valencia Street Circuit,Street circuit,Clockwise,Valencia,Spain,5.419,valencia
74,Watkins Glen International,Race circuit,Clockwise,Watkins Glen,United States,5.430,watkins_glen
75,Yas Marina Circuit,Race circuit,Anti-clockwise,Abu Dhabi,United Arab Emirates,5.281,yas_marina


In [ ]:
circuits_info.drop(['name', 'location'], axis=1, inplace=True)
circuits_info

,type,direction,country,length,circuitId
0,Street circuit,Clockwise,Australia,3.780,adelaide
1,Road circuit,Clockwise,Morocco,7.618,ain-diab
2,Road circuit,Clockwise,United Kingdom,4.828,aintree
3,Street circuit,Clockwise,Australia,5.278,albert_park
4,Race circuit,Clockwise,Portugal,4.653,portimao
...,...,...,...,...,...
71,Race circuit,Part clockwise and part anti-clockwise (figure...,Japan,5.807,suzuka
73,Street circuit,Clockwise,Spain,5.419,valencia
74,Race circuit,Clockwise,United States,5.430,watkins_glen
75,Race circuit,Anti-clockwise,United Arab Emirates,5.281,yas_marina


In [ ]:
# Change column headings
circuits_info.rename(columns={'circuitId': 'circuit_id'}, inplace=True)

In [ ]:
circuits_info

,type,direction,country,length,circuit_id
0,Street circuit,Clockwise,Australia,3.780,adelaide
1,Road circuit,Clockwise,Morocco,7.618,ain-diab
2,Road circuit,Clockwise,United Kingdom,4.828,aintree
3,Street circuit,Clockwise,Australia,5.278,albert_park
4,Race circuit,Clockwise,Portugal,4.653,portimao
...,...,...,...,...,...
71,Race circuit,Part clockwise and part anti-clockwise (figure...,Japan,5.807,suzuka
73,Street circuit,Clockwise,Spain,5.419,valencia
74,Race circuit,Clockwise,United States,5.430,watkins_glen
75,Race circuit,Anti-clockwise,United Arab Emirates,5.281,yas_marina


**Converting Collected data to CSV to looad in another notebook for effecienciency**

In [ ]:
circuits_info.to_csv('circuits.csv', index=False)
weather_info.to_csv('weather.csv', index=False)
races.to_csv('races.csv', index=False)
results.to_csv('results.csv', index=False)
driver_standings.to_csv('driver_standings.csv', index=False)
constructor_standings.to_csv('constructor_standings.csv', index=False)
qualifying_results.to_csv('qualifying.csv', index=False)